In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy
import statistics

import sklearn.linear_model
import sklearn.metrics
import sklearn.naive_bayes
import sklearn.preprocessing
import xgboost

import imblearn
import mrmr
from sklearn.model_selection import cross_val_score

def handle_scale_and_nan(df):
    features = list(df.select_dtypes(include='float64'))
    cat = list(df.select_dtypes(include='object'))
    scaler = sklearn.preprocessing.StandardScaler().fit(df[features])
    df_cont = pd.DataFrame(data=scaler.transform(df[features]), columns=features)
    df_cat = pd.DataFrame(data=df[cat], columns=cat)
    df = pd.concat([df_cat,df_cont],axis=1)
    df = df.fillna(value=0.01)
    
    return df

def split_cats_by_tolerance(
    frame,
    tolerance,
    silent=False,
    randomstate=None,
    split=0.15,
    step=1,
    target="group",
    categories=["Healthy", "AD_MCI", "PD", "PD_MCI_LBD"],
):
    tolerable_list = []
    if randomstate == None:
        randomstate = np.random.randint(0, 2**31)
    elif type(randomstate) == int:
        pass
    while sum(tolerable_list) != 4:
        df_dev, df_test = sklearn.model_selection.train_test_split(
            frame, test_size=split, random_state=randomstate
        )

        dev_dict = dict(df_dev[target].value_counts())
        test_dict = dict(df_test[target].value_counts())

        tolerable_list = []
        stats_dict = {}
        for i in range(0, len(categories)):
            try:
                percents = [
                    (dev_dict[categories[i]] / len(df_dev)),
                    (test_dict[categories[i]] / len(df_test)),
                ]
            except:
                break
            standdev = np.std(percents)
            if standdev <= tolerance:
                tolerable_list.append(1)
                stats_dict[str(categories[i])] = [[*percents], standdev]
            else:
                tolerable_list.append(0)

        randomstate += step

    if sum(tolerable_list) == 4:
        if silent == False:
            print(dev_dict)
            print(test_dict)
            print("Randstate:", randomstate - 1)
            for i in range(0, len(categories)):
                print(
                    "\nPercent",
                    categories[i],
                    "in dev, test:",
                    stats_dict[categories[i]][0],
                    "\nStandard deviation of these values:",
                    stats_dict[categories[i]][1],
                    "\n",
                )
        elif silent == True:
            pass

    return df_dev, df_test

def over_under(df_train,cat_in_excess='Healthy',target='group',randomstate=np.random.randint(0,4294967295)):
    """
    Takes dataframe(s) with only the target value and float64 features
    This function is to balance the samples in an imbalanced training dataset that has one category in excess, with additional categories more near each other
    The categories below the category in excess will be oversampled to equality, then the category in excess will be undersampled to equality
    ---Parameters---
    df_train: the training dataframe
    cat_in_excess: the category which is present in excess, far above the other categories
    target: target column in the dataframe
    randomstate: if chosen, this will the random state for the sampling. Default: None, numpy random integer method between 0 and 4294967295, the range of the sampling module used
    randomstate_sampler: the number of loops to run to compare random states starting from 
    """
    # Drop the excessive category and oversample minority to the intermediate category
    df_train_no_excess = df_train[df_train.group != cat_in_excess]
    over_sampler = imblearn.over_sampling.RandomOverSampler(random_state=randomstate)
    X_train = df_train_no_excess.drop(columns=target)
    y_train = df_train_no_excess[target]
    X_train_over, y_train_over = over_sampler.fit_resample(X_train,y_train)
    df_train_over = pd.concat([y_train_over,X_train_over],axis=1)

    # Re-introduce the excessive category and undersample the majority to the minority
    df_train_excess = pd.concat([df_train_over,df_train[df_train[target] == cat_in_excess]])
    under_sampler = imblearn.under_sampling.RandomUnderSampler(random_state=randomstate)
    X_train = df_train_excess.drop(columns=target)
    y_train = df_train_excess[target]
    X_train_under, y_train_under = under_sampler.fit_resample(X_train,y_train)
    df_train_eq = pd.concat([y_train_under,X_train_under],axis=1)
    
    return df_train_eq

def select_features(data_dev, n):
    X_dev = data_dev.iloc[:, 1:-1]
    y_dev = data_dev.iloc[:, 0]
    
    from sklearn.datasets import make_classification
    X, y = make_classification(n_samples = 100, n_features = 20, n_informative = 2, n_redundant = 2)
    X = pd.DataFrame(X_dev)
    y = pd.Series(y_dev)
    
    from mrmr import mrmr_classif
    feature_list = mrmr_classif(X=X, y=y, K=n)
    print(feature_list)
    
    import json
    aList = feature_list
    jsonStr = json.dumps(aList)
    
    jsonFile = open("mRMR_"+ str(n)+ "_features.json", "w")
    jsonFile.write(jsonStr)
    jsonFile.close()
    
    return feature_list

def apply_ml_model(dev,classifier,scoring_method='balanced_accuracy', cv=10, feature_list=None):
    """
    Finds the score for different ML classifiers
    Takes a dataframe with only target and feature columns
    dev : the development dataframe with a single categorical target column and float64 features
    classifier: ML classifier to test.
        options: "random_forest', "naive_bayes", "decision_tree"
    scoring_method: method of scoring the classifier
        run sklearn.metrics.get_scorer_names() to get a list of scoring methods
    target: target categorical column
    cv: folds to run in evaluation. takes integers or 'max': will run maximum number of folds = # of samples in categories
    """
    if feature_list is None:
        feature_list = selected_features
        
    # define predictor and response variables
    X = dev[feature_list]
    y = dev['group'].values.reshape(-1,1)
    
    # check that the data has equal number of categories in training data
    counts_list = list(dev['group'].value_counts())
    assert len(set(counts_list)) == 1, 'training data should contain equal quantities of categories. run bp_preprocessing.over_under() or other balancer'
    
    
    # choose model based on user input
    if classifier == "random_forest": 
        model = sklearn.ensemble.RandomForestClassifier()
    elif classifier == "naive_bayes":  
        model = sklearn.naive_bayes.GaussianNB()
    elif classifier == "decision_tree": 
        model = sklearn.tree.DecisionTreeClassifier()
    elif classifier == "xgboost": 
        model = XGBClassifier()
    else: 
        print("wrong classifier named entered")
    #define cv quantity:
    if type(cv) == int:
        pass
    elif cv == 'max':
        cv = counts_list[0]
    else:
        raise TypeError('Enter an integer or "max" as a string')
        
    scores = cross_val_score(model, X, y, scoring=scoring_method,cv=cv)
    mean_score = np.mean(abs(scores))
    std = np.std(scores)
    stats_list = [cv, scores, mean_score, std , model, scoring_method, feature_list[-2:]]
    stats_df = pd.DataFrame(data=[stats_list], columns=['folds','scores','abs_avg_score','std','model','scoring_method', 'features'])
    
    return stats_df

In [2]:
dev = pd.read_csv('/Users/katherine/Desktop/data_with_biomarkers.csv',
                       sep='\t')

In [3]:
dev = handle_scale_and_nan(dev)

In [4]:
dev

,group,KV37,LV469,LV861,LVX54,LV746,LV218,LV316,LV312,LV310,...,TEN1,PCDAD,ITM2B,ADSV,A0A1W2PRN1,APOF,DCBD2,LMF2,AB42/AB40,Ttau (pg/ml)
0,AD_MCI,0.196910,1.221120,-0.545899,-0.347466,-0.142571,0.507416,0.125354,-0.170753,0.132798,...,-0.210991,-0.116559,0.010000,1.180216,0.556860,0.010000,0.010000,0.010000,-0.044494,1.787285
1,AD_MCI,0.222919,-0.232289,-0.188719,-0.495320,-0.473139,-0.835324,0.784470,-0.561697,0.448420,...,-1.806582,-0.430047,0.010000,0.010000,0.010000,-0.782416,-1.201640,0.010000,-1.019876,3.003045
2,AD_MCI,-0.776476,1.481940,0.406898,1.081104,0.729230,1.912088,1.478630,2.453163,1.486803,...,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,-0.235850,-0.605881
3,AD_MCI,0.656747,-0.943912,0.053193,-0.346791,0.142617,0.401062,0.563212,0.508889,-0.291028,...,-0.652596,-1.247530,-0.894081,-1.384806,0.534906,0.010000,0.010000,0.010000,-1.235152,0.759451
4,AD_MCI,-0.873193,-0.571888,0.707505,-0.192475,0.789691,0.752272,-0.350730,0.960210,-0.582371,...,0.187345,-0.052730,0.638599,0.010000,0.010000,-0.095173,0.837965,-0.605197,-1.211232,1.507316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,PD_MCI_LBD,-0.999273,-1.081073,-1.370967,-0.135800,-0.981585,-0.568090,-1.033636,-0.461779,-0.742426,...,0.010000,0.657227,-0.233750,-0.480825,0.010000,2.737222,0.010000,-2.411606,1.986002,-0.333582
276,PD_MCI_LBD,-0.959419,1.263972,-0.089190,-2.415437,-0.558657,0.451383,1.060469,-1.059398,-0.661661,...,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,-1.567366,-0.766960
277,PD_MCI_LBD,1.789883,0.777197,0.378536,-2.207599,1.011076,1.694605,1.096632,1.646647,1.285876,...,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,-0.953433,-0.356593
278,PD_MCI_LBD,0.753138,-0.459107,0.133172,0.958462,1.134906,0.419066,-0.496667,-0.410025,-0.589132,...,0.010000,-1.871547,-2.349149,0.453601,0.010000,1.000997,0.010000,-0.272193,-1.697594,-0.655739


In [5]:
train, val = split_cats_by_tolerance(dev,0.01,randomstate=98281)

{'Healthy': 132, 'AD_MCI': 43, 'PD_MCI_LBD': 32, 'PD': 31}
{'Healthy': 24, 'AD_MCI': 8, 'PD_MCI_LBD': 5, 'PD': 5}
Randstate: 98281

Percent Healthy in dev, test: [0.5546218487394958, 0.5714285714285714] 
Standard deviation of these values: 0.008403361344537785 


Percent AD_MCI in dev, test: [0.18067226890756302, 0.19047619047619047] 
Standard deviation of these values: 0.004901960784313722 


Percent PD in dev, test: [0.13025210084033614, 0.11904761904761904] 
Standard deviation of these values: 0.005602240896358551 


Percent PD_MCI_LBD in dev, test: [0.13445378151260504, 0.11904761904761904] 
Standard deviation of these values: 0.007703081232492998 



In [6]:
train = over_under(train,cat_in_excess='Healthy',target='group',randomstate=np.random.randint(0,4294967295))

In [7]:
feature_list = select_features(train, 18)

100%|███████████████████████████████████████████| 18/18 [00:01<00:00, 16.44it/s]

['TAU', 'ITIH5', 'PODN', 'MXRA8', 'POTEJ', '1433G', 'AK1C1', 'PAL4A', 'SPRN', 'FREM2', 'AB42/AB40', 'NEUG', '1433Z', 'BST1', 'HSPB1', 'NAGAB', 'MMP2', 'GDIA']


In [8]:
#Overall score
apply_ml_model(train,classifier='random_forest',scoring_method='balanced_accuracy', cv=10, feature_list = feature_list)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[0.7749999999999999, 0.7749999999999999, 0.462...",0.71625,0.094547,RandomForestClassifier(),balanced_accuracy,"[MMP2, GDIA]"


In [32]:
condition_1 = ['Healthy', 'AD_MCI']
condition_2 = ['Healthy', 'PD']
condition_3 = ['Healthy', 'PD_MCI_LBD']
condition_4 = ['PD', 'PD_MCI_LBD']
condition_5 = ['AD', 'PD']
condition_6 = ['AD', 'PD_MCI_LBD']

train_1 = train[train['group'].isin(condition_1)]
train_2 = train[train['group'].isin(condition_2)]
train_3 = train[train['group'].isin(condition_3)]
train_4 = train[train['group'].isin(condition_4)]
train_5 = train[train['group'].isin(condition_5)]
train_6 = train[train['group'].isin(condition_6)]

In [39]:
#AD vs Healthy
apply_ml_model(train_1,classifier='random_forest',scoring_method='balanced_accuracy', cv=10, feature_list = feature_list)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[0.775, 0.9, 0.9, 0.9, 0.875, 0.775, 1.0, 1.0,...",0.9125,0.083853,RandomForestClassifier(),balanced_accuracy,"[MMP2, GDIA]"


In [40]:
#PD vs. Healthy
apply_ml_model(train_2,classifier='random_forest',scoring_method='balanced_accuracy', cv=10, feature_list = feature_list)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[1.0, 0.9, 0.9, 0.525, 1.0, 1.0, 0.875, 0.75, ...",0.87,0.136382,RandomForestClassifier(),balanced_accuracy,"[MMP2, GDIA]"


In [46]:
#PD_MCI_LBD vs. Healthy
apply_ml_model(train_3,classifier='random_forest',scoring_method='balanced_accuracy', cv=10, feature_list = feature_list)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[0.9, 0.9, 0.875, 0.875, 1.0, 0.875, 1.0, 0.87...",0.9175,0.054829,RandomForestClassifier(),balanced_accuracy,"[MMP2, GDIA]"


In [47]:
#PD vs. PD_MCI_LBD
apply_ml_model(train_4,classifier='random_forest',scoring_method='balanced_accuracy', cv=10, feature_list = feature_list)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[0.775, 0.775, 0.9, 0.8, 1.0, 0.875, 0.875, 0....",0.8375,0.076852,RandomForestClassifier(),balanced_accuracy,"[MMP2, GDIA]"


In [51]:
#AD vs. PD
apply_ml_model(train_5,classifier='random_forest',scoring_method='balanced_accuracy', cv=10, feature_list = feature_list)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,0.0,RandomForestClassifier(),balanced_accuracy,"[MMP2, GDIA]"


In [54]:
#AD vs. PD_MCI_LBD
apply_ml_model(train_6,classifier='random_forest',scoring_method='balanced_accuracy', cv=10, feature_list = feature_list)

,folds,scores,abs_avg_score,std,model,scoring_method,features
0,10,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,0.0,RandomForestClassifier(),balanced_accuracy,"[MMP2, GDIA]"
